In [48]:
pwd

'/home/wsuser/work'

In [49]:
!pip install keras
!pip install tensorflow

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [51]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen =ImageDataGenerator(rescale = 1)

In [52]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_20ac6834bfba4a38a284a4244a15275d = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_20ac6834bfba4a38a284a4244a15275d = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_20ac6834bfba4a38a284a4244a15275d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Yyg4bMEcaIRgTDvY1vLe4MRe0hFws2TxXmg6oBdIjjtu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_20ac6834bfba4a38a284a4244a15275d)

streaming_body_1 = client_20ac6834bfba4a38a284a4244a15275d.get_object(Bucket='plantdiseasedetectionnew-donotdelete-pr-v5qf5xs6hkyvwm', Key='fruit-dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [53]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [54]:
pwd

'/home/wsuser/work'

In [55]:
import os
filenames = os.listdir('/home/wsuser/work/fruit-dataset')

In [56]:
x_test = train_datagen.flow_from_directory('/home/wsuser/work/fruit-dataset/test',
                                            target_size = (128,128),batch_size = 32, class_mode = 'categorical')
x_train =  test_datagen.flow_from_directory('/home/wsuser/work/fruit-dataset/train',
                                           target_size = (128,128),batch_size = 32, class_mode = 'categorical')

Found 1686 images belonging to 6 classes.
Found 5384 images belonging to 6 classes.


In [57]:
print(x_train.class_indices)

{'Apple___Black_rot': 0, 'Apple___healthy': 1, 'Corn_(maize)___Northern_Leaf_Blight': 2, 'Corn_(maize)___healthy': 3, 'Peach___Bacterial_spot': 4, 'Peach___healthy': 5}


In [58]:
model = Sequential()

In [59]:
model.add(Convolution2D(32,(3,3),input_shape = (128,128,3),activation = 'relu'))

In [60]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [61]:
model.add(Flatten())

In [62]:
model.add(Dense(units = 40,kernel_initializer = 'uniform',activation = 'relu'))

In [63]:
model.add(Dense(units = 20,kernel_initializer= 'random_uniform',activation = 'relu'))

In [64]:
model.add(Dense(units = 6,activation = 'softmax',kernel_initializer ='random_uniform'))

In [65]:
model.compile(loss = 'categorical_crossentropy',optimizer = "adam",metrics = ["accuracy"])

In [66]:
model.fit_generator(x_train, steps_per_epoch = 168,epochs = 3,validation_data = x_test,validation_steps = 52)

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
168/168 [==============================] - 64s 379ms/step - loss: 4.1588 - accuracy: 0.4652 - val_loss: 1.7682 - val_accuracy: 0.4796
Epoch 2/3
168/168 [==============================] - 63s 374ms/step - loss: 0.2962 - accuracy: 0.9030 - val_loss: 1.7563 - val_accuracy: 0.3888
Epoch 3/3
168/168 [==============================] - 64s 383ms/step - loss: 0.1146 - accuracy: 0.9692 - val_loss: 1.7545 - val_accuracy: 0.4321


In [67]:
model.save("fruit.h5")

In [68]:
!tar -zcvf image-classification-model_new.tgz fruit.h5

fruit.h5


In [69]:
ls -1

fruit-dataset/
fruit.h5
image-classification-model_new.tgz


In [70]:
!pip install watson-machine-learning-client --upgrade

In [81]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                  "url": "https://us-south.ml.cloud.ibm.com",
                  "apikey": "H6eYoqP1W6s7JCKzT9qJCPVL4pAllEiGC3BraiFAo95H"
                  }
client = APIClient(wml_credentials)

In [82]:
client = APIClient(wml_credentials)

In [83]:
def guid_from_space_name(client, space_name):
        space = client.spaces.get_details()
        return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [86]:
space_uid = guid_from_space_name(client, 'fruittraining')
print("Space UID = " + space_uid)

Space UID = 6ae2542d-a47e-4b62-971b-f9c9c619e5aa


In [87]:
client.set.default_space(space_uid)

'SUCCESS'

In [88]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [89]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [94]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [101]:
model_id

'b00f0692-48a1-40bb-ab74-0d44db9210de'

In [96]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [97]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [98]:
model = load_model('fruit.h5')